#### Note: 이 장에서는 분산 인프라에서 대규모 언어 모델을 훈련하기 위한 대용량 데이터셋과 스크립트를 만듭니다. 따라서 이 노트북의 모든 단계가 코랩이나 캐글 같은 플랫폼에서 실행 가능한 것은 아닙니다. 중요 지점에서 단계를 축소하거나 분산 훈련 스크립트를 만들 때 참고용으로 이 노트북을 사용하세요

#### Note: 이 책의 다른 코드와 달리 이 장의 훈련 코드는 다중 GPU에서 스크립트로 실행하도록 만들어졌습니다. CodeParrot을 직접 훈련하려면 트랜스포머스 저장소(https://oreil.ly/ZyPPR)에 있는 스크립트를 사용하는 것이 좋습니다.

코파일럿, TabNine, Kite 등 트랜스포머를 사용해 코드 자동 완성을 수행하는 애플리케이션들이 있다.  
  
5장에서 GPT 모델을 사용해 고품질 텍스트를 생성하는 방법을 알아보았으므로  
이 장에서는 이 둘을 연결해 파이썬 코드를 생성하는 GPT 유사 모델을 직접 만들어보자.(이 모델을 CodeParrot이라 칭하겠다)

원하는 데이터가 모두 있을 때 무엇을 할 수 있는지 알아보기.  
사전 훈련 단계 자체를 살펴보고, 트랜스포머 모델을 밑바닥부터 훈련하는 법을 배운다.

1. 대용량 데이터셋 수집과 처리  
2. 자신만의 데이터셋을 위한 사용자 정의 토크나이저 만들기  
3. 여러 GPU에서 대규모로 모델 훈련하기

파라미터가 수십억 개인 대규모 모델을 효과적으로 훈련하려면 분산 훈련을 위한 특별한 도구가 필요하다.   
  
분산 훈련은 트랜스포머스의 Trainer도 지원하지만, 지금이야말로 액셀러레이트 Accelerate라는 강력한 파이토치 라이브러리를 소개할 때다.  
이 장의 훈련 코드는 다중 GPU에서 스크립트로 실행해야 한다.

# 10.1 대규모 데이터셋 수집하기

## 10.1.1 대규모 말뭉치 구축의 어려움

사전 훈련된 모델을 사용하면 해당 모델의 토크나이저를 사용해야 한다.  
하지만 다른 도메인의 말뭉치에서 훈련한 토크나이저를 사용하는 것은 대개 최선의 결정이 아니다.  
  
가령 법률문서에서 훈련한 GPT의 사전 훈련 토크나이저를 다른 언어, 음악 악보나 DNA 시퀀스와 같이 완전히 다른 시퀀스에 사용하면 토큰화 결과가 좋지 않다.

In [1]:
from transformers import pipeline, set_seed

generation_gpt = pipeline("text-generation", model="openai-gpt")
generation_gpt2 = pipeline("text-generation", model="gpt2")

In [2]:
def model_size(model):
    return sum(t.numel() for t in model.parameters())
print(f"GPT 크기: {model_size(generation_gpt.model)/1000**2:.1f}M parameters")
print(f"GPT2 크기: {model_size(generation_gpt2.model)/1000**2:.1f}M parameters")

GPT 크기: 116.5M parameters
GPT2 크기: 124.4M parameters


In [3]:
set_seed(1)

In [4]:
def enum_pipeline_outputs(pipe, prompt, num_return_sequences):
    out = pipe(prompt, num_return_sequences=num_return_sequences,
               clean_up_tokenization_spaces=True)
    return "\n".join(f"{i+1}." + s["generated_text"] for i, s in enumerate(out))

In [5]:
zzz = ' '
temp = ["h", "e", "l", "l", "o"]
print(zzz.join(temp))
zzz.join("1" + s for s in temp)

h e l l o


'1h 1e 1l 1l 1o'

In [6]:
prompt = "\nWhen they came back"
print("GPT 자동 완성:\n" + enum_pipeline_outputs(generation_gpt, prompt, 3))
print("")
print("GPT2 자동 완성:\n" + enum_pipeline_outputs(generation_gpt2, prompt, 3))

/home/jj/anaconda3/envs/NLP/lib/python3.8/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (50) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GPT 자동 완성:
1.
When they came back, she 'd want to know what he was going to do. 
 " it 'll teach your brother a lesson, " he said, and then he looked pointedly at my arm and the blood he 'd poured over my skin. "
2.
When they came back, a little boy with a black nose and brown eyes who had started hanging around the neighborhood on special occasions and who lived off of the property. he wore a football jersey with the letters " the black diamond " written on it.
3.
When they came back, and they were still there. " 
 jake nodded. 
 i shrugged and sat back down. " so then, they did come back, " i said, thinking. " but... " 
 " what's her name?

GPT2 자동 완성:
1.
When they came back the body was still on fire, and there were a lot of people who had died. The smell of diesel had been getting worse. No one was doing well. The streets were deserted, with no water and no running water
2.
When they came back to the door, they came back with the fire extinguisher.

The man, who was carrying only a

## 10.1.2 사용자 정의 코드 데이터셋 만들기

파일을 추출하는 과정은 다음과 같이 TransCoder 구현을 참고함  
https://oreil.ly/vih2m  
1. 구글 클라우드 계정을 만듦(무료 크레딧으로 충분)  
2. 계정 아래 구글 빅쿼리 프로젝트를 만듦.  
3. 이 프로젝트에서 데이터셋을 만듦.  
4. 이 프로젝트에서 SQL 요청 결과를 저장할 테이블을 만듦.  
5. github_repos에서 다음 SQL 쿼리를 준비하고 실행한다.(쿼리 결과를 저장하려면 [더보기] > [쿼리 설정]을 선택하고 '쿼리 결과의 대상 테이블 설정'을 체크하고 테이블 이름을 지정한다.)

## 10.1.3 대용량 데이터셋 다루기
작은 컴퓨터에서 대용량 데이터셋을 다룰 때 발생하는 제약 사항을 해결하는 데 데이터셋이 어떤 도움이 되는지 알아보기

컴퓨터의 RAM보다 큰 대용량 데이터셋을 로딩하는 작업은 어려움.  
표준적인 랩톱이나 데스크톱 컴퓨터의 메모리로 로드하기 어렵다.  
  
데이터셋은 이런 문제를 고려해 설계됨.  
#### 메모리와 하드 드라이브 공간의 제약을 해결할 수 있도록 메모리 매핑과 스트리밍(streaming)기능을 제공한다.

## 메모리 매핑

메모리 제약을 극복하기 위해 데이터셋은 zero-copy와 zero-overhead 메모리 매핑을 위한 메커니즘을 사용하며 기본적으로 활성화된다.  
각 데이터셋은 메모리 내용이 바로 반영되는 하나의 파일로 디스크에 캐싱된다.  
데이터셋을 메모리로 로딩하지 않고, 데이터셋은 이 파일에서 읽기 전용 포인터를 열어 메모리 대신 이를 사용한다.  
#### 근본적으로 하드드라이브를 메모리의 확장으로 사용하는 셈이다.

이때까지는 허깅페이스 허브에 있는 원격 데이터셋을 가져올 때 대부분 데이터셋을 사용했지만 여기는 로컬 codeparrot 저장소에 저장된, 50GB의 압축된 JSON 파일을 직접 로드한다.  
먼저 JSON 파일의 압축을 풀어야 한다. -> 하지만 데이터셋이 이를 알아서 처리한다.  
#### 주의할 점은 180GB의 디스크 공간이 필요하다. 다만 RAM은 거의 사용하지 않는다.  
데이터셋의 다운로드 설정에 delete_extracted=True를 지정하면 더 이상 필요하지 않은 모든 파일을 즉시 삭제한다.

In [7]:
from datasets import load_dataset, DownloadConfig

In [8]:
# download_config = DownloadConfig(delete_extracted=True)
# dataset = load_dataset("/mnt/hdd_repo1/codeparrot", split="train",
#                        download_config=download_config)

데이터셋은 내부적으로 압축된 JSON 파일을 최적화된 캐시 파일 하나에 로드해서 내용을 모두 추출하고 읽어들인다.  
이 데이터셋이 로드되면 용량이 얼마나 되는지 보자.

In [9]:
# import psutil # 메모리 추적하는 라이브러리

# print(f"데이터셋에 있는 파이썬 파일의 개수 : {len(dataset)}")
# ds_size = sum(os.stat(f["filename"]).st_size for f in dataset.cache_files)
# # os.stat.st_size는 바이트 단위이므로 GB로 바꿉니다.
# print(f"데이터셋 크기 (캐시 파일) : {ds_size / 2**30:.2f} GB")
# # process.memory_info는 바이트 단위이므로 MB로 바꿉니다.
# print(f"RAM used: {psutil.Process(os.getpid()).memory_info().rss >> 20} MB")

데이터셋은 일반적인 램 메모리 용량보다 훨씬 더 크다.  
하지만 여전히 로드할 수 있고 실제로 사용하는 메모리가 매우 적다.

#### 하지만 전체 데이터셋을 로컬에 저장할 만큼 여유 공간이 충분하지 않다면 어떻게 될까?
다행히 전체 데이터셋을 로컬에 저장하지 않는 방법이 있는데, 바로 데이터셋의 스트리밍이다.

## 스트리밍

(1TB 또는 그 이상 되는) 일부 데이터셋은 표준적인 하드 드라이브가 있더라도 다루기 어렵다.  
이 경우 사용하는 서버의 용량을 높이는 대신 데이터셋을 스트리밍할 수 있다.  
JSONL (JSON Lines), CSV, (원시 포맷 또는 zip, gzip, zstandard 압축된) 텍스트 같이 한 줄씩 읽는 여러 종류의 압축 또는 비압축 파일 포맷을 데이터셋으로 처리할 수 있다.  

캐시 파일을 만들지 않고 압축된 JSON 파일을 바로 데이터셋으로 로드해보자

In [10]:
# streamed_dataset = load_dataset("/mnt/hdd_repo1/codeparrot", split="train", streaming=True)

스트리밍 모드는 압축된 JSON 파일을 열어 동적으로 읽는다.

데이터셋은 이제 IterableDataset 객체이므로 streamed_dataset[1234] 처럼 랜덤하게 원소에 접근할 수는 없지만 대신 next(iter(streamed_dataset)) 같은 방식으로 순서대로 읽어야 한다.  
  
  
shuffle() 같은 메서드는 여전히 사용 가능하지만, 샘플 버퍼를 추출하고 이 버퍼 안에서 랜덤하게 섞는 식으로 동작한다.(버퍼 크기는 조정 가능하다.)  
원시 파일이 여러 개(이 경우. 184개)일 때, shuffle()은 반복마다 파일 순서를 랜덤하게 선택한다.

스트리밍 데이터셋의 샘플은 스트리밍하지 않는 데이터셋의 샘플과 동일하다.

In [11]:
# iterator = iter(streamed_dataset)

# print(dataset[0] == next(iterator))
# print(dataset[1] == next(iterator))

#### 스트리밍 데이터셋의 장점: 데이터셋을 로드할 때 하드 드라이브에 캐시 파일이 생성되지 않고 (많은 양의) 메모리가 필요하지 않다는 것.  
새로운 샘플 배치가 필요할 때 원시 파일을 즉시 추출하고 읽어 들여 해당 배치만 메모리에 로드한다.  
-> 그래서 데이터셋의 메모리 사용량이 크게 줄어든다.

#### 한 단계 더 나아가 로컬 데이터셋이 아니라 허브에 있는 데이터셋을 지정할 수 있다. 그러면 원시 파일을 로컬에 다운로드하지 않고 샘플을 직접 다운로드한다.

In [12]:
remote_dataset = load_dataset("transformersbook/codeparrot", split="train",
                              streaming=True)

Resolving data files:   0%|          | 0/184 [00:00<?, ?it/s]

이 데이터셋은 이전과 완전히 똑같이 동작한다. 하지만 내부적으로 샘플을 동적으로 다운로드한다.  
이렇게 설정하면 (대부분의) 작은 서버에서도 아주 큰 대용량 데이터셋을 사용할 수 있다.  
데이터셋을 훈련 분할과 검증 분할로 나눠 허깅페이스 허브에 업로드하고 스트리밍으로 사용해보자.

In [13]:
# iterator = iter(streamed_dataset)
# print(next(iterator))

In [14]:
iterator2 = iter(remote_dataset)
print(next(iterator2))

{'repo_name': 'ahmedbodi/AutobahnPython', 'path': 'examples/asyncio/websocket/echo/client_coroutines.py', 'copies': '13', 'size': '2044', 'content': '###############################################################################\n##\n##  Copyright (C) 2013-2014 Tavendo GmbH\n##\n##  Licensed under the Apache License, Version 2.0 (the "License");\n##  you may not use this file except in compliance with the License.\n##  You may obtain a copy of the License at\n##\n##      http://www.apache.org/licenses/LICENSE-2.0\n##\n##  Unless required by applicable law or agreed to in writing, software\n##  distributed under the License is distributed on an "AS IS" BASIS,\n##  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n##  See the License for the specific language governing permissions and\n##  limitations under the License.\n##\n###############################################################################\n\nfrom autobahn.asyncio.websocket import WebSocketClientPro

## 10.1.4 허깅페이스 허브에 데이터셋 추가하기

### 데이터셋을 허깅페이스 허브에 업로드하면 다음 작업이 가능해진다.  
1. 훈련 서버에서 쉽게 다운로드한다.  
2. 스트리밍 데이터셋을 허브 데이터셋과 함께 사용한다.  
3. 해당 책의 독자는 물론이고 커뮤니티와 공유한다.

### 데이터셋을 업로드하려면 먼저 터미널에서 다음 명령을 실행하고 인증 정보를 입력해 허깅페이스 계정에 로그인한다.

huggingface-cli login

로그인한 후, 허브에 새로운 데이터셋을 만들고 압축된 JSON 파일을 직접 업로드한다.  
작업을 간단하게 하기 위해 저장소를 두 개 만들기 (하나는 훈련 분할, 다른 하나는 검증 분할을 위한 것)

### 터미널에 repo create 명령을 실행  
~$ huggingface-cli repo create your_dataset_name --type dataset  
  
~$ huggingface-cli repo create your_dataset_name --type dataset --organization your-org-name

### 나는 이렇게 입력함
~$ huggingface-cli repo create codeparrot-train --type dataset  
  
~$ huggingface-cli repo create codeparrot-valid --type dataset  

여기서 저장소 타입을 (가중치를 저장하는 데 사용할 모델 저장소 대신) 데이터셋으로 지정한다.  
또 이 저장소가 속할 조직을 지정한다.  
개인 계정에서 이 명령을 실행한다면 --organization 옵션을 빼도 좋다.

이제 비어 있는 두 저장소를 로컬 컴퓨터에 클론하고, JSON 파일을 각 저장소에 복사하고, 변경 사항을 허브에 푸시해야 한다.  
184개 파일 중 마지막 JSON 압축 파일을 검증 파일로 선택하겠다. (즉, 전체 데이터셋의 약 0.5%다.)

### 다음 명령을 실행해 허브에서 로컬 컴퓨터로 저장소를 클론한다.
Make sure you have git-lfs installed  
https://git-lfs.github.com/  

~$ git lfs install (안되어 있다면)  
  
~$ git clone https://huggingface.co/datasets/namespace/your_dataset_name  
(Here the namespace is either your username or your organization name.)

### 나는 이렇게
~$ git clone https://huggingface.co/datasets/bh8648/codeparrot-train  

~$ git clone https://huggingface.co/datasets/bh8648/codeparrot-valid

### 마지막 깃허브 파일을 제외한 모든 파일을 훈련 세트로 복사한다.

~$ cd codeparrot-train  

~$ sudo cp /mnt/hdd_repo1/codeparrot/*.json.gz .  

~$ rm ./file-000000000183.json.gz

### 그다음 파일을 커밋하고 허브에 푸시한다.

~$ git add .  

~$ git commit -m "Adding dataset files"  

~$ git push

### validation은 마지막 샘플만 선택하여 위와 똑같이 적용

# 10.2 토크나이저 구축하기

사전 훈련된 모델을 사용 -> 사전 훈련을 위해 선택한 전처리 방식을 동일하게 고수 (그렇지 않으면 분포를 벗어난 패턴이나 알 수 없는 토큰이 모델에 입력됨)   
  
새로운 모델을 훈련 -> 기존 토크나이저를 사용하는게 최적이 아닐 수 있음(기존 토크나이저가 훈련한 말뭉치 데이터셋이 광범위한 '불용어 필터링'을 적용했다던가, 프랑스어 텍스트로 훈련하여 평범한 영단어를 인식 못하는 등)

In [15]:
from transformers import AutoTokenizer

def tok_list(tokenizer, string):
    input_ids = tokenizer(string, add_special_tokens=False)["input_ids"]
    return [tokenizer.decode(tok) for tok in input_ids]

tokenizer_T5 = AutoTokenizer.from_pretrained("t5-base")
tokenizer_camembert = AutoTokenizer.from_pretrained("camembert-base")

/home/jj/anaconda3/envs/NLP/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


T5 토크나이저는 불용어 필터링이 광범위하게 적용되어 'sex' 같은 평범한 단어를 본 적이 없음  
CamemBERT 토크나이저는 프랑스어 텍스트로만 훈련되어 'being'같은 평범한 영단어를 인식 못함

In [16]:
print(f'"sex"에 대한 T5 토큰: {tok_list(tokenizer_T5, "sex")}')
print(f'"being"에 대한 CamemBERT 토큰: {tok_list(tokenizer_camembert, "being")}')

"sex"에 대한 T5 토큰: ['', 's', 'ex']
"being"에 대한 CamemBERT 토큰: ['be', 'ing']


이렇게 짧고 평범한 단어를 부분으로 나누면 (문맥 길이가 제한된) 모델에 입력되는 시퀀스 길이가 늘어나서 비효율적이게 된다.  
  
그래서 토크나이저를 훈련하는 데 사용한 데이터셋의 도메인과 전처리 방식을 이해하는 것이 중요하다.

### 토크나이저와 모델은 데이터셋의 편향을 인코딩할 수 있는데, 이는 모델의 후속 행동에 영향을 미친다.
따라서 데이터셋에 맞는 최적의 토크나이저를 얻으려면 토크나이저를 직접 훈련해야 한다.

### ※ 모델의 훈련은 특정 목적하에 작업을 수행할 최적의 모델 가중치 집합을 찾는 과정이다. 그러나 토크나이저 훈련은 역전파나 가중치와 무관하다.
토크나이저 훈련은 텍스트 문자열을 정수 리스트로 매핑하고 모델에 주입할 최적의 매핑을 찾는다.  
  
오늘날 토크나이저에서 최적의 문자열-정수 변환에는, 단위(atomic) 문자열의 리스트로 구성된 어휘사전과 변환, 정규화, 잘라내기, 텍스트 문자열을 인덱스 리스트로 매핑하기 등을 위한 메서드가 사용된다.  
그다음 이 인덱스 리스트가 신경망의 입력이 된다.

## 10.2.1 토크나이저 모델
4장) 토크나이저는 정규화, 사전 토큰화, 토크나이저 모델, 사후 처리 네 단계로 구성된 전처리 파이프라인이다.  
데이터로 훈련하는 토크나이저 파이프라인 부분이 토크나이저 모델이다.  
2장) BPE, WordPeice, 유니그램 같은 부분단어 토큰화 알고리즘을 사용한다.

#### BPE는 기본 단위(단일 문자)의 리스트로 시작해서 가장 자주 함게 등장한 기본 단위를 합쳐 어휘사전에 추가하는 식으로 점진적으로 새 토큰을 만드는 과정을 거쳐 어휘사전을 만든다.  
이 과정은 사전에 정의된 어휘사전 크기에 도달할 때까지 반복된다.

#### 유니그램은 반대로 먼저 말뭉치에 있는 모든 단어와 가능성 있는 부분단어로 기본 어휘사전을 구성한다.  
그다음 어휘사전의 목표 크기에 도달할 때까지 점진적으로 유용성이 떨어지는 토큰을 삭제하거나 분할해 더 작은 어휘사전을 얻는다.  
WordPiece는 유니그램의 전신이며 공식 구현은 구글에서 오픈소스로 공개한 적이 없다.

이런 다양한 알고리즘이 후속 작업의 성능에 미치는 영향은 그 작업에 따라 다르다.  
#### BPE와 유니그램은 대부분의 경우 성능이 꽤 괜찮지만 평가할 때 고려할 점이 있다.

## 10.2.2 토크나이저 성능 측정하기

1. 부분단어 생산력(subword fertility): 토큰화된 단어마다 생성되는 부분단어의 평균 개수를 계산한다.

2. 연속 단어의 비율(proportion of continued words): 말뭉치에서 적어도 두 개의 부분 토큰으로 분할된 토큰화된 단어의 비율

3. 커버리지 측정값(coverage metrics): 토큰화된 말뭉치에서 알 수 없는 단어나 거의 사용되지 않는 비율

때로는 철자 오류나 잡음에 대한 견고성, 도메인 밖 샘플에 대한 모델 성능을 추정하기도 한다.  
(이런 성능은 토큰화 과정의 영향을 크게 받기 때문)

#### 이런 측정값들은 토큰화 성능에 대한 다양한 정보를 제공하지만 토크나이저와 모델의 상호작용을 간과하는 경향이 있다.
예를 들어, 부분단어 생산력은 어휘사전에 가능한 모든 단어를 포함시켜 최소화할 수 있지만, 그러면 모델 입장에서 어휘사전이 매우 커진다.

### 결국 다양한 토큰화 방식의 성능은 일반적으로 후속 모델의 성능을 궁극적인 지표로 사용할 때 가장 잘 추정된다.
예를 들어, 초기 BPE 방식의 성능이 우수하다는 사실을 입증하기 위해 문자나 단어 기반 토큰화 대신 BPE 방식의 토크나이저와 어휘사전을 사용해 훈련한 모델이 기계 번역 작업의 성능을 향상시킴을 보였다.

## 10.2.3 파이썬 코드를 위한 토크나이저

#### 코드 토큰화에 자연어 처리 토크나이저를 사용하는 것이 최적은 아닐 것 같음.  
(공백에 중요한 의미가 있고 줄바꿈에는 별 의미가 없음, 밑줄 문자(_)로 여러 단어를 조합해 변수 이름을 만드는 등)

#### 허깅페이스 허브에서 이 작업에 유용한 토크나이저를 제공하는지 알아보자.  
공백을 유지하는 토크나이저가 필요하니, GPT-2의 토크나이저와 같은 바이트 수준 토크나이저가 좋은 후보가 될 수 있다.  
이 토크나이저를 로드하고 토큰화 방식을 살펴보자

#### ※ 파이썬은 파이썬 코드를 의미 있는 단위(코드 연산, 주석, 들여쓰기, 내어쓰기 등)로 분할하는 tokenize 모듈을 내장하고 있으나 이 토크나이저가 파이썬 기반이라 보통 느리고 파이썬의 GIL(Golobal Interpreter Lock) 때문에 성능이 제한된다.

In [17]:
from transformers import AutoTokenizer

python_code = r"""def say_hello():
    print("Hello, World!")
# Print it
say_hello()
"""
tokenizer = AutoTokenizer.from_pretrained("gpt2")
print(tokenizer(python_code).tokens())

['def', 'Ġsay', '_', 'hello', '():', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġprint', '("', 'Hello', ',', 'ĠWorld', '!"', ')', 'Ċ', '#', 'ĠPrint', 'Ġit', 'Ċ', 'say', '_', 'hello', '()', 'Ċ']


출력이 상당히 이상하다. 이 토크나이저 파이프라인에 있는 다양한 서브 모듈을 실행해서 어떤작업을 수행하는지 알아보자.  
### 먼저 토크나이저에 어떤 정규화가 적용됐는지 본다.

In [18]:
print(tokenizer.backend_tokenizer.normalizer)

None


GPT-2 토크나이저는 정규화를 사용하지 않는다.  
어떤 정규화 단계도 거치지 않고 원시 유니코드 입력을 바로 사용함. 

### 다음으로 사전 토큰화를 확인

In [19]:
print(tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(python_code))

[('def', (0, 3)), ('Ġsay', (3, 7)), ('_', (7, 8)), ('hello', (8, 13)), ('():', (13, 16)), ('ĊĠĠĠ', (16, 20)), ('Ġprint', (20, 26)), ('("', (26, 28)), ('Hello', (28, 33)), (',', (33, 34)), ('ĠWorld', (34, 40)), ('!")', (40, 43)), ('Ċ', (43, 44)), ('#', (44, 45)), ('ĠPrint', (45, 51)), ('Ġit', (51, 54)), ('Ċ', (54, 55)), ('say', (55, 58)), ('_', (58, 59)), ('hello', (59, 64)), ('()', (64, 66)), ('Ċ', (66, 67))]


#### 토큰과 함께 출력된 숫자
-> 토크나이저는 문자열과 토큰 사이를 전환하는 데 매우 유용한 오프셋 트랭킹(offset tracking)기능이 있다.  
  
-> 입력 문자열에 대한 모든 연산이 추적되기 때문에 토큰화 이후에 토큰이 입력 문자열의 어떤 부분에 해당하는지 정확하게 알 수 있다.  
  
-> 이 숫자는 단순히 토큰이 유래된 원본 문자열의 위치를 나타낸다.  
  
-> 예를 들어 첫째 줄의 단어 'hello'는 원본 문자열의 인덱스 8과 13 사이에 있다. 일부 문자가 정규화 단계에서 삭제되더라도 각 토큰을 원본 문자열의 해당 부분에 연결할 수 있다.

#### Ċ, Ġ  처럼 기이한 문자.
-> 바이트 수준이란 표현은 이 토크나이저가 유니코드 문자가 아닌 바이트 단위로 동작함을 뜻한다.  
  
각 유니코드 문자는 문자에 따라 1에서 4바이트로 구성된다.  
유니코드 알파벳에는 143,859개의 유니코드 문자가 있지만 바이트 알파벳에는 256개만 있다는 것이 바이트의 장점이다.  
따라서 유니코드 문자를 이런 바이트의 시퀀스로 변환할 수 있다.  
  
만약 바이트를 사용하면 UTF-8로 구성된 어떤 문자열도 256개 값의 알파벳으로 구성된 더 긴 문자열로 표현 가능하다.  
즉, 256개 알파벳만 사용해 어떤 유니코드 문자열도 처리하는 모델이 생긴다.  
  
일부 문자의 바이트 표현을 확인해보자.

In [20]:
a, e = u"a", u"€"
byte = ord(a.encode("utf-8"))
print(f'`{a}`는 단일 바이트 {a.encode("utf-8")}로 인코딩됩니다: {byte}')
byte = [ord(chr(i)) for i in e.encode("utf-8")]
print(f'`{e}`는 세 바이트 {e.encode("utf-8")}로 인코딩됩니다: {byte}')

`a`는 단일 바이트 b'a'로 인코딩됩니다: 97
`€`는 세 바이트 b'\xe2\x82\xac'로 인코딩됩니다: [226, 130, 172]


gpt-2는 256개 입력 바이트를 출력 가능한 표준 유니코드 문자에 해당하는 유니코드 문자열로 매핑한 다음 BPE 알고리즘을 적용한다.

In [21]:
from transformers.models.gpt2.tokenization_gpt2 import bytes_to_unicode

byte_to_unicode_map = bytes_to_unicode()
unicode_to_byte_map = dict((v, k) for k, v in byte_to_unicode_map.items())
base_vocab = list(unicode_to_byte_map.keys())

print(f'기본 어휘 사전 크기: {len(base_vocab)}')
print(f'첫 번째 원소: `{base_vocab[0]}`, last element: `{base_vocab[-1]}`')

기본 어휘 사전 크기: 256
첫 번째 원소: `!`, last element: `Ń`


#### ※ NLP의 표준 Byte-Pair Encoding(BPE)은 이름과 달리 대개 유니코드 문자열에서 작동한다.  
#### ※ Byte-Level(바이트 수준) BPE는 바이트에서 동작하는 신형 BPE다.  
유니코드 문자열을 바이트로 읽는다면 간단한 BPE 부분단어 분할 알고리즘의 재사용이 가능하다.

In [22]:
# BPE 문자 매핑의 예
import pandas as pd
from transformers.models.gpt2.tokenization_gpt2 import bytes_to_unicode

byte_to_unicode_map = bytes_to_unicode()
unicode_to_byte_map = dict((v, k) for k, v in byte_to_unicode_map.items())
base_vocab = list(unicode_to_byte_map.keys())

examples = [
    ['Regular characters', '`a` and `?`', f'{ord("a")} and {ord("?")}' , f'`{byte_to_unicode_map[ord("a")]}` and `{byte_to_unicode_map[ord("?")]}`'],
    ['Nonprintable control character (carriage return)', '`U+000D`', f'13', f'`{byte_to_unicode_map[13]}`'],
    ['A space', '` `', f'{ord(" ")}', f'`{byte_to_unicode_map[ord(" ")]}`'],
    ['A nonbreakable space', '`\\xa0`', '160', f'`{byte_to_unicode_map[ord(chr(160))]}`'],
    ['A newline character', '`\\n`', '10', f'`{byte_to_unicode_map[ord(chr(10))]}`'],
]

pd.DataFrame(examples, columns = ['Description', 'Character', 'Bytes', 'Mapped bytes'])

,Description,Character,Bytes,Mapped bytes
0,Regular characters,`a` and `?`,97 and 63,`a` and `?`
1,Nonprintable control character (carriage return),`U+000D`,13,`č`
2,A space,` `,32,`Ġ`
3,A nonbreakable space,`\xa0`,160,`ł`
4,A newline character,`\n`,10,`Ċ`


In [23]:
print(tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(python_code))

[('def', (0, 3)), ('Ġsay', (3, 7)), ('_', (7, 8)), ('hello', (8, 13)), ('():', (13, 16)), ('ĊĠĠĠ', (16, 20)), ('Ġprint', (20, 26)), ('("', (26, 28)), ('Hello', (28, 33)), (',', (33, 34)), ('ĠWorld', (34, 40)), ('!")', (40, 43)), ('Ċ', (43, 44)), ('#', (44, 45)), ('ĠPrint', (45, 51)), ('Ġit', (51, 54)), ('Ċ', (54, 55)), ('say', (55, 58)), ('_', (58, 59)), ('hello', (59, 64)), ('()', (64, 66)), ('Ċ', (66, 67))]


#### Ċ는 줄바꿈, Ġ는 공백을 표시한 것을 알 수 있다.
#### 공백, 특히 연속된 공백이 보존된다. (예를 들어, ĊĠĠĠ는 공백 세 개를 의미한다)
#### 연속된 공백은 단어 하나로 간주한다.
#### 단어 앞 공백은 후속 단어에 포함된 단어의 일부로 간주한다. (가령 Ġsay)

### 그럼 BPE 알고리즘으로 실험해보자. 이미 언급했듯 미리 정의된 어휘사전 크기에 도달할 때까지 단어를 부분으로 나눈다.
GPT-2 토크나이저의 어휘사전은 50,257개 단어로 구성된다.  
기본 어휘사전은 256개 바이트 값에 해당한다.  
50,000개 추가 토큰은 가장 자주 함께 등장한 토큰을 반복적으로 합쳐 만든다.  
문서 경계를 나타내는 특별한 문자가 어휘사전에 추가된다.

#### 이 내용을 토크나이저의 길이 속성에서 쉽게 확인해보자.

In [24]:
print(f"어휘사전의 크기: {len(tokenizer)}")

어휘사전의 크기: 50257


샘플 입력 코드에 전체 파이프라인을 실행하면 다음과 같은 결과를 얻는다.

In [25]:
print(tokenizer(python_code).tokens())

['def', 'Ġsay', '_', 'hello', '():', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġprint', '("', 'Hello', ',', 'ĠWorld', '!"', ')', 'Ċ', '#', 'ĠPrint', 'Ġit', 'Ċ', 'say', '_', 'hello', '()', 'Ċ']


BPE 토크나이저는 대부분 단어를 유지한다.  
하지만 들여쓰기에 있는 여러 공백을 몇 개의 연속된 공백으로 나눈다.  
(코드에서 훈련되지 않았기 때문, 대부분 텍스트에서 연속된 공백은 드물다.)  
### 이러한 경우가 토크나이저 모델이 주어진 데이터셋의 도메인에 잘 맞지 않는 한 예다.
### -> 앞서 논의했듯 타깃 말뭉치에서 토크나이저를 재훈련하는 것이 해결책이다.

## 10.2.4 토크나이저 훈련하기
이 예제의 말뭉치에서 바이트 수준 BPE 토크나이저를 재훈련하겠다.  
#### 트랜스포머스가 제공하는 토크나이저는 간단하게 재훈련되는데, 다음 내용이 필요하다.
1. 목표 어휘사전의 크기를 지정한다.  
2. 토크나이저 모델을 훈련하기 위해 입력 문자열을 공급할 반복자(iterator)를 준비한다.  
3. train_new_from_iterator() 메서드를 호출한다.

#### 딥러닝 모델은 훈련 말뭉치에서 특정 세부 내용을 많이 기억하도록 훈련하지만,
#### 토크나이저는 주요 통곗값을 추출하도록 훈련한다.  
-> 간단히 말해 토크나이저는 말뭉치에서 가장 자주 등장한 문자 조합을 알아내는 훈련을 한다.

In [26]:
tokens = sorted(tokenizer.vocab.items(), key=lambda x: len(x[0]), reverse=True)

In [27]:
for t, _ in tokens[:8]:
    print(t+'\n')

ÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤ

Ġ=================================================================

Ġ----------------------------------------------------------------


ÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤ

________________________________________________________________

----------------------------------------------------------------

................................................................



#### 위는 온라인 포럼에서 사용하는 구분선으로 보인다.  
#### 이번에는 가장 드물게 나타나 어휘사전의 마지막에 등록된 단어를 확인해보자.

In [28]:
tokens = sorted(tokenizer.vocab.items(), key=lambda x: x[1], reverse=True)
print([f'{tokenizer.convert_ids_to_tokens(i)}' for t, i in tokens[:12]]);

['<|endoftext|>', 'Ġgazed', 'Ġinformants', 'ĠCollider', 'Ġregress', 'ominated', 'Ġamplification', 'Compar', 'âĢ¦."', 'Ġ(/', 'Commission', 'ĠHitman']


### 첫 토큰 <|endoftext|>은 텍스트 시퀀스의 끝을 지정할 때 사용하는 특수 토큰으로, BPE 어휘사전이 구축된 후 추가된다.
모델은 각 토큰에 관련된 단어 임베딩을 학습해야 하는데, 임베딩 행렬에 잡음 단어가 많이 포함되어 있다면 좋지 않을 것.  
또한 세상의 시공간적 지식(가령 Hitman과 Commission 같은 고유 명사)을 어휘사전에 있는 벡터와 함께 별개의 토큰으로 부여해 저수준에서 이를 임베딩할 수 있다.

#### BPE 토크나이저가 이런 토큰을 생성한다면 목표 어휘사전이 너무 크거나 말뭉치에 특수 토큰이 포함됐다는 신호가 된다.

### 예제의 말뭉치에서 새 토크나이저를 훈련하고 학습된 어휘사전을 살펴보겠다.
데이터셋의 통계를 대표할 말뭉치가 필요하니 말뭉치에서 약 100,000개 문서 (1~2GB 데이터)를 선택하겠다.

In [29]:
base_vocab

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '|',
 '}',
 '~',
 '¡',
 '¢',
 '£',
 '¤',
 '¥',
 '¦',
 '§',
 '¨',
 '©',
 'ª',
 '«',
 '¬',
 '®',
 '¯',
 '°',
 '±',
 '²',
 '³',
 '´',
 'µ',
 '¶',
 '·',
 '¸',
 '¹',
 'º',
 '»',
 '¼',
 '½',
 '¾',
 '¿',
 'À',
 'Á',
 'Â',
 'Ã',
 'Ä',
 'Å',
 'Æ',
 'Ç',
 'È',
 'É',
 'Ê',
 'Ë',
 'Ì',
 'Í',
 'Î',
 'Ï',
 'Ð',
 'Ñ',
 'Ò',
 'Ó',
 'Ô',
 'Õ',
 'Ö',
 '×',
 'Ø',
 'Ù',
 'Ú',
 'Û',
 'Ü',
 'Ý',
 'Þ',
 'ß',
 'à',
 'á',
 'â',
 'ã',
 'ä',
 'å',
 'æ',
 'ç',
 'è',
 'é',
 'ê

In [30]:
tokenizer

GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True)

### 시간 좀 오래 걸림

In [31]:
#hide_output
from tqdm.auto import tqdm

length = 100000
dataset_name = 'transformersbook/codeparrot-train'
dataset = load_dataset(dataset_name, split="train", streaming=True)
iter_dataset = iter(dataset)

def batch_iterator(batch_size=10):
    for _ in tqdm(range(0, length, batch_size)):
        yield [next(iter_dataset)['content'] for _ in range(batch_size)]

new_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(), 
                                                  vocab_size=12500,
                                                  initial_alphabet=base_vocab)

Repo card metadata block was not found. Setting CardData to empty.


Resolving data files:   0%|          | 0/183 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

AutoTokenizer.train_new_from_iterator()는 사용 중인 토크나이저가 "빠른(fast)" 토크나이저인 경우에만 작동합니다. 다음 섹션에서 볼 수 있듯이 🤗Transformers 라이브러리에는 두 가지 유형의 토크나이저가 포함되어 있습니다. 한 유형은 순수하게 Python으로 작성되어 있고 다른 유형(빠른 토크나이저)은 🤗Tokenizers 라이브러리의 도움을 받아서 Rust 프로그래밍 언어로 작성된 토크나이저입니다.  
  
트랜스포머스에는 기존에 존재하는 것들과 동일한 특성을 가진 새로운 토크나이저를 학습하는데 사용할 수 있는 매우 간단한 API가 있습니다. 바로 AutoTokenizer.train_new_from_iterator()가 그것입니다. 

### 이 어휘사전이 작업과 얼마나 관련됐는지 알아보기 위해 BPE 알고리즘이 만든 첫 단어와 마지막 단어를 조사해보자.
256바이트는 건너뛰고 그 다음에 추가된 첫 토큰을 살펴본다.

In [32]:
tokens = sorted(new_tokenizer.vocab.items(), key=lambda x: x[1], reverse=False)

In [33]:
tokens

[('<|endoftext|>', 0),
 ('!', 1),
 ('"', 2),
 ('#', 3),
 ('$', 4),
 ('%', 5),
 ('&', 6),
 ("'", 7),
 ('(', 8),
 (')', 9),
 ('*', 10),
 ('+', 11),
 (',', 12),
 ('-', 13),
 ('.', 14),
 ('/', 15),
 ('0', 16),
 ('1', 17),
 ('2', 18),
 ('3', 19),
 ('4', 20),
 ('5', 21),
 ('6', 22),
 ('7', 23),
 ('8', 24),
 ('9', 25),
 (':', 26),
 (';', 27),
 ('<', 28),
 ('=', 29),
 ('>', 30),
 ('?', 31),
 ('@', 32),
 ('A', 33),
 ('B', 34),
 ('C', 35),
 ('D', 36),
 ('E', 37),
 ('F', 38),
 ('G', 39),
 ('H', 40),
 ('I', 41),
 ('J', 42),
 ('K', 43),
 ('L', 44),
 ('M', 45),
 ('N', 46),
 ('O', 47),
 ('P', 48),
 ('Q', 49),
 ('R', 50),
 ('S', 51),
 ('T', 52),
 ('U', 53),
 ('V', 54),
 ('W', 55),
 ('X', 56),
 ('Y', 57),
 ('Z', 58),
 ('[', 59),
 ('\\', 60),
 (']', 61),
 ('^', 62),
 ('_', 63),
 ('`', 64),
 ('a', 65),
 ('b', 66),
 ('c', 67),
 ('d', 68),
 ('e', 69),
 ('f', 70),
 ('g', 71),
 ('h', 72),
 ('i', 73),
 ('j', 74),
 ('k', 75),
 ('l', 76),
 ('m', 77),
 ('n', 78),
 ('o', 79),
 ('p', 80),
 ('q', 81),
 ('r', 82),
 

In [34]:
for x in [t for t, _ in tokens[257:280]]:
    temp = new_tokenizer.convert_tokens_to_string(list(x))
    print(repr(temp))

'  '
'    '
'   '
'        '
'se'
'in'
'       '
're'
'on'
'te'
'\n       '
'\n        '
'or'
'st'
'de'
'\n   '
'th'
'le'
' ='
'lf'
'self'
'me'
'al'


#### 마지막 단어를 확인하기

In [35]:
for x in [t for t, _ in tokens[-12:]]:
    temp = new_tokenizer.convert_tokens_to_string(list(x))
    print(repr(temp))

' capt'
' embedded'
' regarding'
'Bundle'
'355'
' recv'
' dmp'
' vault'
' Mongo'
' possibly'
'implementation'
'Matches'


토크나이저가 어떻게 동작하는지 보기 위해 간단한 파이썬 예제 코드를 토큰화하기

In [36]:
print(new_tokenizer(python_code).tokens())

['def', 'Ġs', 'ay', '_', 'hello', '():', 'ĊĠĠĠ', 'Ġprint', '("', 'Hello', ',', 'ĠWor', 'ld', '!")', 'Ċ', '#', 'ĠPrint', 'Ġit', 'Ċ', 's', 'ay', '_', 'hello', '()', 'Ċ']


#### 프로그램 키워드는 아니지만 토크나이저가 World나 say 같은 평범한 영단어를 분할한다.  
-> 이런 단어가 말뭉치에 자주 등장하리라 예상되는데 마음에 좀 걸린다.  
-> 파이썬의 예약어가 모두 어휘사전에 있는지 확인해보자

In [37]:
import keyword

print(f'파이썬 전체 예약어 개수: {len(keyword.kwlist)}')
for keyw in keyword.kwlist:
    if keyw in keyword.kwlist:
        if keyw not in new_tokenizer.vocab:
            print(f"예약어 '{keyw}'는 어휘 사전에 없습니다.")

파이썬 전체 예약어 개수: 35
예약어 'await'는 어휘 사전에 없습니다.
예약어 'finally'는 어휘 사전에 없습니다.
예약어 'nonlocal'는 어휘 사전에 없습니다.


#### finally 같이 매우 자주 등장하는 예약어가 없다.  
#### 따라서 데이터셋에서 더 많은 샘플을 가져와 더 큰 어휘사전을 만들겠다.  
#### 어휘사전을 32,768개 단어로 구성하고(8의 배수가 GPU/TPU 계산에 더 효율적이다.)  
#### 토크나이저를 더 많은 말뭉치 데이터에서 훈련하겠다.

In [38]:
length = 200000
new_tokenizer_larger = tokenizer.train_new_from_iterator(batch_iterator(),
                                                         vocab_size=32768,
                                                         initial_alphabet=base_vocab)

  0%|          | 0/20000 [00:00<?, ?it/s]

더 많은 문서를 추가해도 가장 자주 등장한 토큰이 크게 바뀌리라고 기대하진 않는다.  
하지만 마지막 토큰을 확인해보자.

In [39]:
tokens = sorted(new_tokenizer_larger.vocab.items(), 
                key=lambda x: x[1],
                reverse=False)

In [40]:
for x in [t for t, _ in tokens[-12:]]:
    temp = new_tokenizer_larger.convert_tokens_to_string(list(x))
    print(repr(temp))

" '<?"
'Functional'
' Images'
'encoders'
' bibrec'
' OPTIONAL'
' rdclass'
'SocketAddressTag'
'资金'
'DEPLOYMENT'
'经纪公司代码'
")'],"


#### 새 토크나이저로 샘플 코드를 토큰화하겠다.

In [41]:
print(new_tokenizer_larger(python_code).tokens())

['def', 'Ġsay', '_', 'hello', '():', 'ĊĠĠĠ', 'Ġprint', '("', 'Hello', ',', 'ĠWorld', '!")', 'Ċ', '#', 'ĠPrint', 'Ġit', 'Ċ', 'say', '_', 'hello', '()', 'Ċ']


#### 여기서도 편리하게 들여쓰기가 어휘사전에 유지됐으며 Hello, World, say 같은 평범한 영단어도 하나의 토큰으로 포함된다.  
모델이 후속 작업에서 데이터를 처리할 때 기대할 수 있는 결과와 잘 맞는 것 같다.

앞에서와 같이 어휘사전에 없는 파이썬 예약어를 조사해보자.

In [42]:
for keyw in keyword.kwlist:
    if keyw not in new_tokenizer_larger.vocab:
        print(f"예약어 '{keyw}'는 어휘사전에 없습니다.")

예약어 'nonlocal'는 어휘사전에 없습니다.


여전히 어휘사전에 nonlocal 예약어가 없는데, 이 단어는 구문을 복잡하게 만들어서 실제로도 드물게 사용된다.  
따라서 어휘사전에 포함시키지 않는 것이 합당하다(고 책에서는 말함).

수동으로 조사해보니 새로운 토크나이저가 이 작업에 잘 맞을 것 같다고 한다.  
하지만 앞서 언급한 대로 모델의 성능을 측정하지 않고 토크나이저의 성능을 객관적으로 평가하기는 어렵다.  
이 토크나이저를 사용해 모델을 훈련한 후에 실제로 얼마나 잘 동작하는지 알아보자.

※ 토큰화된 코드 샘플의 시퀀스 길이를 비교해서 새 토크나이저가 기본 GPT-2 토크나이저보다 거의 두 배 더 효율적임을 쉽게 확인했다.  
  
새 토크나이저는 기존 토크나이저가 텍스트를 인코딩할 때 사용한 토큰의 약 절반만 사용했다.  
이로 인해 아무런 비용을 들이지 않고 실제적인 모델의 문맥 크기가 두 배로 늘어난다.  
문맥 윈도 크기 1,024에서 새로운 토크나이저로 모델을 훈련하는 것은 문맥 윈도 크기 2,048에서 기존 토크나이저로 모델을 훈련하는 것과 동일하다.  
  
하지만 훨씬 더 빠르고 메모리 효율성도 더 높다는 이점이 있다.

## 10.2.5 허브에 사용자 정의 토크나이저 저장하기

토크나이저를 훈련했으니 저장해야 한다.  
토크나이저를 저장하고 나중에 어디에서든 가져다 쓸 수 있게 하려면 허깅페이스 허브에 업로드해야 함.

In [43]:
# from transformers import AutoModel
# from transformers import AutoTokenizer

# # Load model and tokenizer
# model = AutoModel.from_pretrained('업로드할_모델_경로')
# tokenizer = AutoTokenizer.from_pretrained('업로드할_모델_경로')

# # Huggingface Access Token
# ACCESS_TOKEN = '액세스_토큰_붙여넣기'

# # Upload to Huggingface
# model.push_to_hub('Huggingface_Repo_이름', use_temp_dir=True, use_auth_token=ACCESS_TOKEN)
# tokenizer.push_to_hub('Huggingface_Repo_이름', use_temp_dir=True, use_auth_token=ACCESS_TOKEN)

In [44]:
# 터미널에서 허깅페이스에 로그인 한 뒤

model_ckpt = "codeparrot"
org = "bh8648"
new_tokenizer_larger.push_to_hub(model_ckpt, organization=org)

/home/jj/anaconda3/envs/NLP/lib/python3.8/site-packages/transformers/utils/hub.py:652: UserWarning: The `organization` argument is deprecated and will be removed in v5 of Transformers. Set your organization directly in the `repo_id` passed instead (`repo_id={organization}/{model_id}`).
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/bh8648/codeparrot/commit/6fdacbe89b012a13e4deb6b3bcfa41699a104f54', commit_message='Upload tokenizer', commit_description='', oid='6fdacbe89b012a13e4deb6b3bcfa41699a104f54', pr_url=None, pr_revision=None, pr_num=None)

어떤 조직 내에 푸시하고 싶지 않다면 organization 매개변수를 빼면 된다.  
그러면 자신의 네임스페이스 안에 codeparrot이란 이름의 저장소가 만들어지고 누구든지 다음 코드를 실행해 이 토크나이저를 로드할 수 있다.

In [45]:
# reloaded_tokenizer = AutoTokenizer.from_pretrained('bh8648/codeparrot')
reloaded_tokenizer = AutoTokenizer.from_pretrained(org + '/' + model_ckpt)
print(reloaded_tokenizer(python_code).tokens())

['def', 'Ġsay', '_', 'hello', '():', 'ĊĠĠĠ', 'Ġprint', '("', 'Hello', ',', 'ĠWorld', '!")', 'Ċ', '#', 'ĠPrint', 'Ġit', 'Ċ', 'say', '_', 'hello', '()', 'Ċ']


#### 허브에 저장된 어휘사전과 파일을 조사할 수도 있다.
나중에 재현할 수 있도록 새 토크나이저를 조금만 재훈련시켰던 new_tokenizer도 저장해보자.

In [46]:
new_tokenizer.push_to_hub(model_ckpt + "-small-vocabulary", organization=org)

CommitInfo(commit_url='https://huggingface.co/bh8648/codeparrot-small-vocabulary/commit/414f02f6e4632a6c5629542387fc8f3e7de5a4a3', commit_message='Upload tokenizer', commit_description='', oid='414f02f6e4632a6c5629542387fc8f3e7de5a4a3', pr_url=None, pr_revision=None, pr_num=None)

# 10.3 밑바닥부터 모델을 훈련하기

### 코잘 언어 모델링 causal language modeling
-> 우리가 하려는 작업은 코드 샘플 시작 부분을 모델에게 제공하고 코드의 나머지 부분을 생성해 완성하라고 요청하는 것이다.  

-> 레이블이 없는 데이터셋을 사용하는 자기 지도 훈련 목표. = 코잘 언어 모델링  
  
-> 코드 자동 완성은 이와 직접적으로 관련된 후속 작업임.(이런 작업에는 GPT 계열 같은 디코더 전용 아키텍처가 가장 잘 맞다.)
#### ※ 코잘 언어 모델링에서 모델은 마스킹된 미래 토큰을 예측한다.

### 마스크드 언어 모델링
-> 관련성이 있지만 조금 다른 작업은 모델에게 잡음이 섞인 코드 샘플(랜덤한 단어나 마스킹된 단어로 코드 명령을 바꾸는 등)을 주고 깨끗한 원본 샘플을 재구성하라고 요청하는 것
  
-> 이것도 자기 지도 훈련 목표이며, 일반적으로 마스크드 언어 모델링masked language modeling 또는 잡음 제거 목표denoising objective라 한다.  
  
-> 많은 모델이 이런 식으로 사전 훈련되며, 레이블링된 샘플이 제한적인 후속 작업의 경우 대규모 말뭉치에서 훈련된 마스크드 언어 모델을 사용해 모델을 미세 튜닝할 수 있다.
#### ※ 마스크드 언어 모델링에서는 입력 토큰 중 일부가 마스킹되거나 바뀌며 이후 원본 토큰을 예측하는 것이 모델의 작업이다.  

### 시퀀스-투-시퀀스 훈련
-> 정규식 같은 수동 규칙으로 주석이나 독스트링을 코드에서 분리해서 레이블링된 데이터셋으로 사용하도록 (코드, 주석) 쌍의 대규모 데이터셋을 구축하는 작업.  
  
-> 훈련은 한 카테고리(코드나 주석)를 모델의 입력으로 사용하고 다른 카테고리(주석이나 코드)를 레이블로 사용하는 지도 학습 목표가 된다.  
  
-> 즉, (입력, 레이블) 쌍을 사용하는 지도 학습 문제가 된다.

-> 이런 지도 학습 훈련 작업에 직접적으로 관련된 후속 작업은 입력/출력을 어떻게 지정하느냐에 따라 코드로부터 문서를 생성하거나 문서로부터 코드를 생성하는 일이다.  
  
-> 이런 설정에서 한 시퀀스는 다른 시퀀스로 변환되며 T5, BART, PEGASUS 같은 인코더-디코더 모델이 여기에 잘맞다.
#### ※ 시퀀스-투-시퀀스 작업을 위한 인코더-디코더 아키텍처에서 입력은 경험적인 규칙을 사용해 주석/코드 쌍으로 분할되고, 모델은 주석과 코드 중 하나를 입력으로 받아 다른 하나를 생성한다.

### 코드 자동 완성 모델을 구축하고 싶기 때문에 첫 번째 훈련 목표와 GPT 아키텍처를 선택한다. 이제 새로운 GPT-2 모델을 생성해보자.

## 10.3.2 모델 초기화

from_pretrained() 메서드로 모델을 로드하지 않고 새 모델을 만든다.  
  
하지만 gpt2-xl 설정과 동일한 하이퍼파라미터를 사용하고 새 토크나이저를 위해 어휘사전의 크기만 바꾼다.  

그다음 이 설정을 from_config() 메서드에 사용해 새 모델을 초기화한다.

In [47]:
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer

org = 'bh8648'
model_ckpt = "codeparrot"
tokenizer = AutoTokenizer.from_pretrained(org + '/' + model_ckpt)
config = AutoConfig.from_pretrained("gpt2-xl", vocab_size=len(tokenizer))
model = AutoModelForCausalLM.from_config(config)

In [48]:
len(tokenizer)

32768

In [49]:
tokenizer

GPT2TokenizerFast(name_or_path='bh8648/codeparrot', vocab_size=32768, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True)

In [50]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(32768, 1600)
    (wpe): Embedding(1024, 1600)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-47): 48 x GPT2Block(
        (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1600, out_features=32768, bias=False)
)

wte: word token embedding  
wpe: word position embedding

### 모델이 얼마나 큰지 확인해보기

In [51]:
print(f"GPT-2 (xl) 크기: {model_size(model) / 1000**2:.1f}M parameters")

GPT-2 (xl) 크기: 1529.6M parameters


15억 개 파라미터가 있다. 용량이 크지만 준비한 데이터셋도 대용량이다.  
보통 대규모 언어 모델은 데이터셋이 충분히 크기만 하면 효율적인 훈련이 가능하다.  
새로 초기화한 모델을 models/ 폴더에 저장하고 허브에 푸시해보자.

In [52]:
model.save_pretrained("models/" + model_ckpt, push_to_hub=True,
                      organization=org)

체크포인트 크기가 5GB보다 크기 때문에 모델을 허브에 푸시하는 데 몇 분이 걸릴 수 있다.  
모델이 매우 크기 때문에 모든 것이 잘 동작하는지 확인하기 위해 작은 버전도 만들겠다.  
표준 GPT-2 크기를 기본으로 사용한다.

In [53]:
model_ckpt

'codeparrot'

In [54]:
tokenizer = AutoTokenizer.from_pretrained(org + '/' + model_ckpt)
config_small = AutoConfig.from_pretrained("gpt2", vocab_size=len(tokenizer))
model_small = AutoModelForCausalLM.from_config(config_small)
print(f"GPT-2 크기: {model_size(model_small)/1000**2:.1f}M개의 파라미터")

GPT-2 크기: 111.0M개의 파라미터


#### 이 모델도 쉽게 공유하고 재사용하기 위해 허브에 저장하기

In [55]:
model_small.save_pretrained("models/" + model_ckpt + "-small", push_to_hub=True,
                             organization=org)

두 개의 모델을 준비했으니, 이제 훈련할 때 효율적으로 입력 데이터를 주입할 방법이 필요하다.

## 10.3.3 데이터로더 구축하기
훈련 효율을 극대화하도록 문맥 크기를 꽉 채운 시퀀스를 모델에 제공한다.  
  
예를 들어 모델의 문맥 크기가 1,024개 토큰이라면 훈련할 때 매번 1,024개 토큰의 시퀀스를 제공하는 것이 좋다.  
하지만 일부 코드 샘플은 토큰의 개수가 1,024개보다 더 짧거나 길다.  

이 경우 모델에게 sequence_length 길이 시퀀스의 배치를 주입하기 위해 시퀀스 마지막을 자르거나 패딩해야 한다.    
하지만 그러면 훈련의 효율성이 조금 떨어지고 패딩된 토큰의 '레이블'을 '패딩'하고 '마스킹'해야 한다.  
-> 데이터 제약에 비해 컴퓨팅 제약이 훨씬 더 커진다.  
  
따라서 여기서는 시퀀스 마지막 부분을 너무 많이 잃지 않도록 하는 쉽고 효율적인 방법을 사용하려 한다.  
1. 여러 샘플을 토큰화한 다음 EOS 토큰으로 이를 연결해 매우 긴 시퀀스를 만든다.  
2. 마지막으로 이 시퀀스를 동일한 크기의 청크chunk로 나눈다.  

-> 이 방법을 사용하면 마지막 데이터에서 손실되는 부분이 미미하다.

#### 예를 들어 입력 문자열의 문자 개수를 다음과 같이 정의해서 토큰화된 샘플에 약 100개의 완전한 시퀀스가 있게 만들 수 있다.

In [56]:
number_of_sequences = 100    # 토크나이저로부터 얻으려는 (잘린) 시퀀스의 개수(가령 100)
sequence_length = 1024       # 토크나이저가 반환한 각 시퀀스의 토큰 개수 (가령 1024)
characters_per_token = 500  # 사전에 추정해야 하는 각 출력 토큰의 평균 문자 개수

input_characters = number_of_sequences * sequence_length * characters_per_token

In [57]:
input_characters # 토크나이저에 입력된 문자열에 있는 문자의 개수

51200000

input_characters 개의 문자로 된 문자열을 입력하면 평균적으로 number_of_sequences 개의 출력 시퀀스를 얻는다.  
number_of_sequence = 100은 시퀀스를 약 100개 쌓고 기껏해야 너무 짧거나 긴 마지막 원소를 잃는다는 의미다. 즉, 데이터셋의 1%를 잃는 셈.  
동시에 이 방식은 대부분 파일의 끝을 잘라내지 않아 편향을 일으키지 않는다.

#### 먼저 데이터셋에 있는 토큰의 평균 문자 길이를 예측해보자.

In [58]:
examples, total_characters, total_tokens = 500, 0, 0
dataset = load_dataset("transformersbook/codeparrot-train", split="train",
                       streaming=True)

Repo card metadata block was not found. Setting CardData to empty.


Resolving data files:   0%|          | 0/183 [00:00<?, ?it/s]

In [59]:
for _, example in tqdm(zip(range(examples), iter(dataset)), total=examples):
    total_characters += len(example["content"])
    total_tokens += len(tokenizer(example["content"]).tokens())

characters_per_token = total_characters / total_tokens

print(characters_per_token)

  0%|          | 0/500 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2599 > 1024). Running this sequence through the model will result in indexing errors


3.6231516195736053


#### 평균적으로 각 토큰당 약 3.6의 character를 가진다.  

모델에게 일정한 길이의 입력을 주입하도록 사용자 정의 IterableDataset (파이토치에서 제공하는 헬퍼 클래스)를 만들기 위해 필요한 것이 모두 준비됐다.  
  
#### IterableDataset을 상속해서 방금 살펴본 로직을 기반으로 다음 원소를 반환하는 \__iter__() 함수를 작성해보기.

In [60]:
import torch
from torch.utils.data import IterableDataset

class ConstantLengthDataset(IterableDataset):
    
    def __init__(self, tokenizer, dataset, seq_length=1024, num_of_sequences=1024, 
                 chars_per_token=3.6):
        self.tokenizer = tokenizer
        self.concat_token_id = tokenizer.eos_token_id
        self.dataset = dataset
        self.seq_length = seq_length
        self.input_characters = seq_length * chars_per_token * num_of_sequences
    
    def __iter__(self):
        iterator = iter(self.dataset)
        more_examples = True
        while more_examples:
            buffer, buffer_len = [], 0
            while True:
                if buffer_len >= self.input_characters:
                    m=f"Buffer full: {buffer_len}>={self.input_characters:.0f}"
                    print(m)
                    break
                try:
                    m=f"Fill buffer: {buffer_len}<{self.input_characters:.0f}"
                    print(m)
                    buffer.append(next(iterator)["content"])
                    buffer_len += len(buffer[-1])
                except StopIteration:
                    iterator = iter(self.dataset)
            
            all_token_ids = []
            tokenized_inputs = self.tokenizer(buffer, truncation=False) # truncation은 문장 잘림을 허용할지 말지 정하는 것
            for tokenized_input in tokenized_inputs['input_ids']:
                all_token_ids.extend(tokenized_input + [self.concat_token_id]) # 여기가 모든 시퀀스를 토큰화 한 다음 각 시퀀스의 끝에 EOS 토큰만 붙이고 하나로 쭉 연결하는 작업
            
            for i in range(0, len(all_token_ids), self.seq_length): # 여기가 모두 이어 붙인 시퀀스(토큰화되어있음)를 chunk에 맞춰 구분짓는 곳
                input_ids = all_token_ids[i : i + self.seq_length]
                if len(input_ids) == self.seq_length:
                    yield torch.tensor(input_ids)

\__iter__() 메서드는 충분한 개수의 문자를 포함할 때까지 문자열 버퍼를 채운다.  
  
버퍼 안의 모든 원소는 토큰화되어 EOS 토큰으로 연결되고 그다음 all_token_ids에 있는 긴 시퀀스가 seq_length 크기로 나뉜다.  
  
일반적으로 패딩된 가변 길이 시퀀스를 쌓을 때는 훈련 시 패딩을 무시하도록 어텐션 마스크가 필요하다.  
여기서는 동일한 (최대) 길이 시퀀스만 제공하므로 마스크가 필요하지 않으며 input_ids만 반환하면 된다.  
  
이 사용자 정의 데이터셋을 테스트해보자.

In [61]:
shuffled_dataset = dataset.shuffle(buffer_size=100) # 입력된 buffer_size 만큼 data를 채우고(처음부터 순서대로) 무작위로 섞음
constant_length_dataset = ConstantLengthDataset(tokenizer, shuffled_dataset,
                                                num_of_sequences=10)
dataset_iterator = iter(constant_length_dataset)

lengths = [len(b) for _, b in zip(range(5), dataset_iterator)] # 5개만 시퀀스 길이 꺼내보기
print(f"시퀀스 길이: {lengths}")

Fill buffer: 0<36864
Fill buffer: 3806<36864
Fill buffer: 5182<36864
Fill buffer: 6297<36864
Fill buffer: 7366<36864
Fill buffer: 11481<36864
Fill buffer: 17562<36864
Fill buffer: 22827<36864
Fill buffer: 29806<36864
Buffer full: 44530>=36864
시퀀스 길이: [1024, 1024, 1024, 1024, 1024]


의도대로 작동해 모델에 전달할 일정 길의의 입력을 얻었다.  
이제 모델을 위해 신뢰할만한 데이터 소스를 갖췄으니 실제 훈련 루프를 만들어보자.

#### ※ ConstantLengthDataset을 만들기 전에 원시 데이터셋을 섞었다.  
IterableDataset 이므로 처음에 전체 데이터셋을 섞을 수 없다.  
대신 데이터셋에서 원소를 가져오기 전에 buffer_size 크기의 버퍼를 할당하고 버퍼 안의 원소를 섞는다.

## 10.3.4 훈련 루프 정의하기

사용자 정의 언어 모델을 훈련할 때 분명한 제약 조건은 GPU의 메모리 제한이다.  
이 예제에서는 여러 개의 GPU를 훈련에 사용하기 위해 '데이터 병렬화(data parallelism)'을 구현한다.  
  
다행히 액셀러레이트를 사용해 예제 코드를 확장할 수 있다.  
액셀레레이트 라이브러리는 분산훈련을 위해 하드웨어를 쉽게 바꿀 수 있게 고안됐다.  
분산 훈련을 위해 Trainer를 사용할 수도 있지만, 액셀러레이트를 사용하면 훈련 루프를 완전하게 제어할 수 있으니 여기서 살펴보자.

액셀러레이트는 혼합 정밀도와 어떤 종류의 분산 환경(단일 GPU, 다중 GPU, TPU)에서도 훈련 스크립트를 실행할 수 있는 간편한 API를 제공한다.  
동일한 코드를 디버깅하기 위해 로컬 컴퓨터에서 실행하거나 최종 훈련을 위해 대규모 훈련 클러스터에서도 실행이 가능하다.  
  
기본 파이토치 훈련 루프에서 약간만 변경하면 된다.

In [76]:
import torch
import torch.nn.functional as F
from datasets import load_dataset
from accelerate import Accelerator

import torch
import torch.nn.functional as F
from datasets import load_dataset
from accelerate import Accelerator

accelerator = Accelerator()
model = torch.nn.Transformer()
optimizer = torch.optim.Adam(model.parameters())
dataset = load_dataset('my_dataset')
data = torch.utils.data.DataLoader(dataset, shuffle=True)

model, optimizer, data = accelerator.prepare(model, optimizer, data)

model.train()
for epoch in range(10):
    for source, targets in data:
        optimizer.zero_grad()
        output = model(source)
        loss = F.cross_entropy(output, targets)
        accelerator.backward(loss)
        optimizer.step()

#### 바뀐 부분 중 핵심은 prepare() 메서드 호출이다.
모델, 옵티마이저, 데이터로더를 모두 준비하고 인프라에 분산한다.  
이렇게 파이토치 훈련루프를 조금 바꿔 다양한 인프라로 훈련을 확장한다.  
  
이 점을 유념하면서 훈련 스크립트를 만들고 몇 개의 헬퍼 함수를 정의하겠다.
먼저 훈련을 위한 하이퍼파라미터를 설정하고 접근이 용이하도록 Namespace로 감싼다.

In [62]:
from argparse import Namespace

# 작은 모델에 해당하는 파라미터
config = {"train_batch_size": 2, # 12
          "valid_batch_size": 2, # 12
          "weight_decay": 0.1,
          "shuffle_buffer": 1000,
          "learning_rate": 2e-4, # 5e-4
          "lr_scheduler_type": "cosine",
          "num_warmup_steps": 750, # 2000
          "gradient_accumulation_steps": 16, # 1
          "max_train_steps": 50000, # 150000
          "max_eval_steps": -1,
          "seq_length": 1024,
          "seed": 1,
          "save_checkpoint_steps": 50000} # 15000

args = Namespace(**config)

In [63]:
args

Namespace(gradient_accumulation_steps=16, learning_rate=0.0002, lr_scheduler_type='cosine', max_eval_steps=-1, max_train_steps=50000, num_warmup_steps=750, save_checkpoint_steps=50000, seed=1, seq_length=1024, shuffle_buffer=1000, train_batch_size=2, valid_batch_size=2, weight_decay=0.1)

### 그다음 훈련을 위핸 로깅을 설정한다.  
모델을 밑바닥부터 훈련하기 떄문에 훈련 시간이 조금 걸리고 고가의 인프라가 필요하다.  
따라서 관련된 모든 정보를 저장하고 쉽게 참조할 수 있게 만드는 것이 좋다.  
  
setup_logging() 메서드는 세 개의 로깅 수준을 설정한다.  
표준 파이썬 Logger(https://oreil.ly/P9Xrm), 텐서보드 (https://oreil.ly/kY5ri),  
wandb(https://oreil.ly/BCC3k) 를 사용한다.  
주어진 문제와 선호하는 바에 따라 로깅 프레임워크를 추가하거나 삭제할 수 있다.

In [66]:
from torch.utils.tensorboard import SummaryWriter
import logging
import wandb

def setup_logging(project_name):
    logger = logging.getLogger(__name__)
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S", level=logging.INFO, handlers=[
            logging.FileHandler(f"log/debug_{accelerator.process_index}.log"),
            logging.StreamHandler()])
    if accelerator.is_main_process: # 로깅을 한 번만 설정한다.
        wandb.init(project=project_name, config=args)
        run_name = wandb.run.name
        tb_writer = SummaryWriter()
        tb_writer.add_hparams(vars(args), {'0':0})
        logger.setLevel(logging.INFO)
        datasets.utils.logging.set_verbosity_debug()
        transformers.utils.logging.set_verbosity_info()
    else:
        tb_writer = None
        run_name = ''
        logger.setLevel(logging.ERROR)
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()
    
    return logger, tb_writer, run_name

각 워커(worker)는 고유한 accelerator.process_index를 받는다.  
이를 FileHandler에 사용해 각 워커의 로그를 개별 파일에 기록한다.  
  
또 메인 워커에서만 true인 accelerator.is_main_process 속성도 사용한다.  
이를 사용해 텐서보드와 wandb의 로거logger가 여러 번 초기화되지 않게 하고 다른 워커에서 로깅 수준을 낮춘다.  
나중에 허브에서 실험 결과에 이름을 부여하기 위해 자동으로 생성된 고유한 wandb.run.name을 반환한다.

#### 텐서보드와 wandb에 측정값을 기록할 함수도 정의해보자.
여기서도 accelerator.is_main_process를 다시 사용해 측정값이 워커마다 저장되지 않고 한 번만 기록되게 한다.

In [67]:
def log_metrics(step, metrics):
    logger.info(f"Step {step}: {metrics}")
    if accelerator.is_main_process:
        wandb.log(metrics)
        [tb_writer.add_scalar(k, v, step) for k, v in metrics.items()]

#### 그다음 사용자 정의 ConstantLanguageDataset 클래스로 훈련 세트와 검증 세트를 위한 데이터로더를 만드는 함수를 작성한다.

In [72]:
from torch.utils.data.dataloader import DataLoader

def create_dataloaders(dataset_name):
    train_data = load_dataset(dataset_name+'-train', split="train", streaming=True)
    train_data = train_data.shuffle(buffer_size=args.shuffle_buffer, seed=args.seed)
    
    valid_data = load_dataset(dataset_name+'-valid', split="validation", streaming=True)
    
    train_dataset = ContantLengthDataset(tokenizer, train_data, seq_length=args.seq_length)
    valid_dataset = ContantLengthDataset(tokenizer, valid_data, seq_length=args.seq_length)

    train_dataloader = DataLoader(train_dataset, batch_size=args.train_batch_size)
    eval_dataloader = DataLoader(valid_dataset, batch_size=args.valid_batch_size)
    
    return train_dataloader, eval_dataloader

마지막에 배치 처리를 위해 DataLoader로 감싼다.  
액셀러레이트가 배치를 각 워커로 분산해줄 것이다.

또 하나 구현할 것은 최적화다.
#### 메인 루프에서 옵티마이저와 학습률을 설정하겠지만, 가중치 감쇠(weight decay)를 받아야 하는 파라미터를 구분하기 위해 헬퍼 함수를 정의하겠다.  
보통 절편과 LayerNorm의 가중치에는 가중치 감쇠를 적용하지 않는다.

In [73]:
def get_gouped_params(model, no_decay=["bias", "LayerNorm.weight"]):
    params_with_wd, params_without_wd = [], []
    for n, p in model.named_parameters(): # 이름(name)과 파라미터(param)를 반환
        if any(nd in n for nd in no_decay):
            params_without_wd.append(p)
        else:
            params_with_wd.append(p)
    
    return [{'params': params_with_wd, 'weight_decay': args.weight_decay},
            {'params': params_without_wd, 'weight_decay': 0.0}]

#### 이따금 검증 세트에서 모델을 평가해야 한다.
따라서 평가 세트에서 손실과 복잡도(perplexity)를 계산하는 평가 함수를 추가한다.

In [74]:
def evaluate():
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch, labels=batch)
        loss = outputs.loss.repeat(args.valid_batch_size)
        losses.append(accelerator.gather(loss))
        if args.max_eval_steps > 0 and step >= args.max_eval_steps: break
    loss = torch.mean(torch.cat(losses))
    try:
        perplexity = torch.exp(loss)
    except OverflowError:
        perplexity = torch.tensor(float("inf"))
    return loss.item(), perplexity.item()

복잡도는 모델의 출력 확률 분포가 얼마나 타깃 토큰을 잘 예측하는지 측정한다.  
복잡도가 낮을수록 성능이 더 좋다.  
복잡도는 모델 출력에서 얻은 크로스 엔트로피 손실에 지수 함수를 적용해 계산한다.  
특히 훈련 초기에 손실이 높을 때 복잡도를 계산하면 수치적으로 오버플로가 발생할 수 있다.  
이 오류를 캐치해서 이 경우 복잡도를 무한대로 설정한다.

훈련 스크립트에 이를 모두 적용하기 전에 추가로 사용할 기능이 하나 있다.  
허깅페이스 허브는 모델과 데이터셋을 저장하고 버전을 관리하기 위해 내부적으로 깃을 사용한다.  
huggingface_hub 라이브러리의 Repository를 사용해 프로그래밍적으로 저장소에 접근하고 풀, 브랜치, 커밋, 푸시를 할 수 있다.  
  
#### 스크립트에 이를 이용해 훈련하는 동안 모델 체크포인트를 지속적으로 허브에 푸시할 것.

### 이제 헬퍼 함수가 모두 준비됐으니 훈련 스크립트의 핵심 부분을 작성해보자.

In [78]:
set_seed(args.seed)

# 엑셀러레이트
accelerator = Accelerator()
samples_per_step = accelerator.state.num_processes * args.train_batch_size

# 로깅
logger, tb_writer, run_name = setup_logging(project_name.split("/")[1])
logger.info(accelerator.state)

# 모델과 토크나이저를 로드한다.
if accelerator.is_main_process:
    hf_repo = Repository("./", clone_from=project_name, revision=run_name)
model = AutoModelForCausalLM.from_pretrained("./", gradient_checkpointing=True)
tokenizer = AutoTokenizer.from_pretrained("./")

# 데이터셋과 데이터로더를 로드한다.
train_dataloader, eval_dataloader = create_dataloaders(dataset_name)

# 옵티마이저와 학습률 스케줄러를 준비한다.
optimizer = AdamW(get_grouped_params(model), lr=args.learning_rate)
lr_scheduler = get_scheduler(name=args.lr_scheduler_type, optimizer=optimizer,
                             num_warmup_steps=args.num_warmup_steps,
                             num_training_steps=args.max_train_steps)

def get_lr():
    return optimizer.params_groups[0]['lr']

# 'accelerator'로 모든 것을 준비한다.(매개변수 순서는 중요하지 않습니다.)
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

# 모델을 훈련한다.
model.train()
completed_steps = 0
for step, batch in enumerate(train_dataloader, start=1):
    loss = model(batch, labels=batch).loss
    log_metrics(step, {'lr':get_lr(),
                       'samples': step*samples_per_step,
                       'steps': completed_steps,
                       'loss/train': loss.item()})
    loss = loss / args.gradient_accumulation_steps
    accelerator.backward(loss)
    if step % args.gradient_accumulation_steps == 0:
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        completed_steps += 1
    
    if step % args.save_checkpoint_steps == 0:
        logger.info("Evaluating and saving model checkpoint")
        eval_loss, perplexity = evaluate()
        log_metrics(step, {'loss/eval': eval_loss, 'perplexity': perplexity})
        accelerator.wait_for_everyone()
        unwrapped_model = accelerator.unwrap_model(model)
        if accelerator.is_main_process:
            unwrapped_model.save_pretrained("./")
            hf_repo.push_to_hub(commit_message=f"step {step}")
        model.train()
    if completed_steps >= args.max_train_steps:
        break

# 마지막 체크포인트를 평가하고 저장한다.
logger.info("Evaluating and saving model after training")
eval_loss, perplexity = evaluate()
log_metrics(step, {"loss/eval": eval_loss, 'perplexity': perplexity})
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrapped_model(model)
if accelerator.is_main_process:
    unwrapped_model.save_pretrained("./")
    hf_repo.push_to_hub(commit_message=f"final model")

NameError: name 'project_name' is not defined

### 코드가 꽤 길지만, 분산 인프라에서 대규모 언어 모델을 훈련할 때 필요한 코드 전부다. 이 스크립트를 조금씩 나눠서 가장 중요한 부분을 설명하겠다.

### 모델저장
-> 모델 저장소 내에서 이 스크립트를 실행하면 스크립트가 시작되면서 wandb에서 얻은 run_name을 따라 새 브랜치를 체크아웃한다.  
-> 나중에 체크포인트마다 모델을 커밋하고 허브에 푸시한다.  
-> 이런 방식으로 실험할 때마다 새 브랜치가 만들어지고 각 커밋은 모델 체크포인트에 해당한다.  
-> 모델을 저장할 때 제대로 동기화되도록 wait_fro_everyone()과 unwrap_model()을 호출한다.

### 최적화
-> 모델 최적화를 위해 선형적인 워밍업 단계를 거친 후, 코사인 학습률 스케쥴러와 함께 AdamW를 사용한다.  
-> 하이퍼파라미터의 경우 GPT-3 논문에 기술된 비슷한 크기의 모델에서 사용한 파라미터와 비슷하게 설정함.

### 평가
-> 모델을 저장할 때마다 평가 세트에서 평가한다.  
-> 즉 각 save_checkpoint_steps마다, 그리고 훈련이 끝난 후에 평가한다.  
-> 검증 손실과 함께 검증 복잡도도 기록한다.

### 그레디언트 누적과 체크포인팅
-> 최신 GPU에서 실행하더라도 GPU 메모리가 필요한 배치 크기에 맞지 않는다.  
-> 따라서 몇번의 역방향 패스에서 그레디언트를 모아 그레디언트가 충분히 누적됐을 때 최적화 단계를 수행하는 그레디언트 누적을 구현한다.  
-> Trainer를 사용해 이를 수행하는 방법은 6장에서 보았다.  
  
-> 대규모 모델의 경우 단일 배치조차도 하나의 GPU에 맞지 않다.  
-> 이 경우 '그레디언트 체크포인팅 gradient checkpointing'이란 방법을 사용해 훈련 속도를 약 20% 낮추면서 메모리 사용량을 줄일 수 있다. (https://oreil.ly/94oj1)  
-> 이를 사용하면 더 큰 모델도 단일 GPU에서 훈련할 수 있다.

#### 다중 GPU에서 모델을 훈련하는 방법 -> 그중에서도 액셀러레이트에서 사용하는 방법
DataDistributedParallelism(DDP)  
https://oreil.ly/m4iNm  
단일 GPU에 맞는 것보다 더 큰 배치에서 모델을 빠르게 훈련한다는 장점이 있다.

#### 모델이 하나의 GPU에 들어갈 수 없을 때는 좀 더 정교한 병렬화 전략이 필요하다.
https://oreil.ly/3uhfq -> 허깅페이스에서 문서가 삭제된거 같은데 parallelism 파트를 더 찾아봐야할듯?

## 10.3.5 훈련 실행
훈련 서버에서 실행되도록 훈련 스크립트를 codeparrot_training.py 파일에 저장하겠다.  
이 스크립트와 함게 필요한 파이썬 패키지가 모두 들어있는 파일을 허브의 모델 저장소에 추가하겠다.  
허브에 있는 모델은 근본적으로 깃 저장소이므로 이 저장소를 클론하고 원하는 어떤 파일을 추가해 허브에 다시 푸시할 수 있다.  
  
훈련 서버에서 다음과 같은 몇 개의 명령으로 훈련을 시작한다.

\$ git clone https://huggingface.co/transformersbook/codeparrot  
  
\$ cd codeparrot  
  
\$ pip install -r requirements.txt  

\$ wandb login  
  
\$ accelerate config  
  
\$ accelerate launch codeparrot_training.py  

이게 전부다.  
wandb login 명령을 실행하면 로깅을 위해 wandb 계정을 인증해야 한다.  
accelerate config 명령은 인프라 설정 과정을 안내해준다.

#### 전체 훈련 실행이 성공적으로 완료된 후 다음 명령으로 이 실험 브랜치를 메인 브랜치로 머지해 허브로 푸시한다.

$ git checkout main  
  
$ git merge <RUN_NAME>
  
$ git push

RUN_NAME은 머지하려는 허브의 실험 브랜치 이름이어야 한다.  
이제 훈련된 모델이 있으므로 성능을 조사하는 방법을 알아보자.

# 10.4 결과 및 분석

정성적인 분석과 정량적인 분석을 수행한다.

#### 정성적인 분석
구체적인 샘플을 사용해서 모델의 성공 사례와 실패 사례를 더 잘 이해하게 된다.

#### 정량적인 분석
대규모 테스트 세트에서 통계적으로 모델의 성능을 평가한다.

#### 먼저 샘플 몇 개를 살펴보고 모델을 체계적이고 안정적으로 평가하는 방법을 논의한다.  
파이프라인으로 작은 모델을 감싸고 이어서 샘플 코드 입력을 전달해보자.

In [79]:
from transformers import pipeline, set_seed

model_ckpt = 'transformersbook/codeparrot-small'
generation = pipeline("text-generation", model=model_ckpt, device=0)

주어진 프롬프트에서 완성된 코드 후보를 생성하기 위해 생성 파이프라인을 사용한다.  
기본적으로 이 파이프라인은 사전에 정의된 길이에 도달할 떄까지 코드를 생성한다.  
출력은 여러 개의 함수나 클래스를 포함할 수 있다.  
따라서 출력을 간략하게 유지하기 위해 정규식으로 첫째로 등장한 함수나 클래스를 추출하는 first_block() 함수를 구현하겠다.  
그 아래 complete_code() 함수는 이 로직을 적용하고 CodeParrot이 생성한 코드 완성을 출력한다.

In [83]:
import re
from transformers import set_seed

def first_block(string):
    return re.split('\nclass|\ndef|\n#|\n@|\nprint|\nif', string)[0].rstrip()

def complete_code(pipe, prompt, max_length=64, num_completions=4, seed=1):
    set_seed(seed)
    gen_kwargs = {"temperature": 0.4, "top_p":0.95, "top_k":0, "num_beams":1, "do_sample":True}
    code_gens = generation(prompt, num_return_sequences=num_completions, max_length=max_length, **gen_kwargs)
    code_strings = []
    for code_gen in code_gens:
        generated_code = first_block(code_gen["generated_text"][len(prompt):])
        code_strings.append(generated_code)
    print(('\n' + '='*80 + '\n').join(code_strings))

#### 간단한 샘플로 모델이 사각형의 면적을 계산하는 함수를 작성하도록 하겠다.

In [84]:
prompt = '''def area_of_rectangle(a: float, b: float):
    """Return the area of the rectangle."""'''
complete_code(generation, prompt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



    return math.sqrt(a * b)

    return a * b / 2.0

    return a * b / 2.0

    return a / b


생성된 코드가 모두 맞지는 않지만 이중에 정답이 있다.

#### 그럼 모델이 HTML에서 URL을 추출하는 조금 더 어려운 작업도 시켜보자.

In [85]:
prompt = '''def get_urls_from_html(html):
    """Get all embedded URLs in a HTML string."""'''
complete_code(generation, prompt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



    if not html:
        return []
    return [url for url in re.findall(r'<a href="(/[^/]+/[^"]+?)">', html)]

    return [url for url in re.findall(r'<a href="(.*?)" class="url">(.*?)</a>', html)]

    if not html:
        return []
    return [url for url in re.findall(r'<a href="([^"']+)">', html)]

    return [url for url in re.findall(r'<a href="(.*?)"', html)]


#### 허깅페이스 홈페이지로 이 함수를 테스트해보자.

In [94]:
import requests

def get_urls_from_html(html):
    return [url for url in re.findall(r'<a href="(.*?)"', html)]

print("|".join(get_urls_from_html(requests.get('https://hf.co/').text)))
# https://로 시작하는 URL은 외부페이지, 나머지는 웹사이트 내 하위 페이지여야 하는데 외부페이지 부분이 안나타나네 흠...

/models|/spaces/HuggingFaceH4/open_llm_leaderboard|/spaces/facebook/seamless_m4t|/spaces/multimodalart/LoraTheExplorer|/spaces/HuggingFaceM4/idefics_playground|/spaces/fffiloni/Image-to-Story|/spaces|/datasets|/docs/transformers|/docs/diffusers|/docs/safetensors|/docs/huggingface_hub|/docs/tokenizers|/docs/peft|/docs/transformers.js|/docs/timm


#### 마지막으로 순수한 파이썬 함수를 넘파이를 사용하는 함수로 바꿀 수 있는지 알아보자
다음 코드 블록에서 대용량 GPT-2 체크포인트를 메모리에 로드합니다. 코랩이나 캐글 같은 플랫폼에서는 램이나 GPU 메모리가 부족하기 때문에 인스턴스가 종료될 수 있습니다. 이런 경우 model_ckpt = "transformersbook/codeparrot-small"로 바꾸어 작은 체크포인트를 사용하면 이 예제를 실행할 수 있습니다.

In [95]:
model_ckpt = "transformersbook/codeparrot"
generation = pipeline('text-generation', model=model_ckpt, device=0)

prompt = '''# a function in native python:
def mean(a):
    return sum(a)/len(a)

# the same function using numpy:
import numpy as np
def mean(a):'''

complete_code(generation, prompt, max_length=64)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



    return np.mean(a)

    return sum(a)/len(a)

    return np.mean(a)

    return a/len(a)


#### 이번에는 CodeParrot 모델이 사이킷런 모델도 만들 수 있는지 알아보자.

In [96]:
prompt = '''X = np.random.randn(100, 100)
y = np.random.randint(0, 1, 100)

# fit random forest classifier with 20 estimators'''
complete_code(generation, prompt, max_length=96)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



reg = DummyRegressor()

forest = RandomForestClassifier(n_estimators=20)

forest.fit(X, y)

clf = ExtraTreesClassifier(n_estimators=100, max_features='sqrt')
clf.fit(X, y)

clf = RandomForestClassifier(n_estimators=20)
clf.fit(X, y)

regr_1 = DecisionTreeClassifier(max_depth=4)

regr_2 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=4),
                          n_estimators=300, random_state=0)

regr_1.fit


두 번째 경우는 ExtraTreesClassifier를 훈련하려고 시도했으며,  
네 번째 경우는 2개의 DecisionTree 모델(하나는 DecisionTree, 하나는 AdaBoost)을 서로 섞으려고 한듯하다.  
다른 경우는 요청한 대로 코드를 생성했다.(두 번째와 네 번째도 랜덤포레스트와 비슷한 역할을 하긴 함)

#### 생성된 텍스트의 품질을 측정하는 방법은 5장에서 알아봤다.  
#### 그중 하나로 많이 사용되는 BLEU 지표는 일반적인 한계가 있기도 하지만, 이 문제에는 특히 잘 맞지 않다.

BLEU 점수는 참조 텍스트와 생성된 텍스트 간의 n-gram 중복을 측정한다.  
코드를 작성할 때 변수와 클래스에 자우도가 많고 프로그램에 일관성이 있다면, 이름을 짓는 방식으로 성공 여부가 결정되지 않는다.  
하지만 BLEU는 참조 코드에 있는 이름과 다른 이름으로 생성한 결과에 불리하다.  
(사실 이런 이름은 거의 예측하기가 불가능하다. (실제 프로그래머도 예측 못함))

#### 소프트웨어 개발 분야에는 유닛 테스트(unit test)와 같이 코드 품질을 측정할 때 신뢰할 만한 좋은 방법이 있다.  
이런 방법으로 OpenAI Codex 모델을 평가했다.  
코딩 작업을 위해 생성한 여러 코드를 일련의 단위 테스트를 통해 실행하고 테스트를 통과한 비율을 계산한다.  
이 장에서 만든 모델의 성능을 제대로 측정하려면 동일한 평가 방식을 적용해야 하겠지만, 이 내용은 이 장의 범위를 넘어선다.  
자세한 내용은 CodeParrot이 HumanEval 벤치마크에서 어떻게 동작하는지를 다룬 블로그 포스트  
https://oreil.ly/hKOP8 을 참고할 것.


# 10.5 결론

이 장에서 수행한 일을 정리해보자.

여기서는 파이썬 코드 자동완성 함수를 만들어보았다.

먼저 대규모 언어 모델을 사전 훈련하기 위해 대규모 데이터셋을 직접 구축했다.  
  
그다음 이 데이터셋으로 파이썬 코드를 효율적으로 인코딩하는 사용자 정의 토크나이저를 만들었다.  
  
마지막으로 액셀러레이트를 사용해 모든 것을 연결하고 훈련 스크립트를 작성해서 200줄 미만의 코드로 다중 GPU 인프라에서 소규모와 대규모 GPT-2 모델을 밑바닥부터 훈련했다.  
  
모델 출력을 조사하면서 모델이 적절하게 이어지는 코드를 생성할 수 있음을 보았고 모델을 체계적으로 평가하는 방법을 논의했다.

이제 허브에 있는 많은 pretrained model을 fine-tuning 하는 법만이 아니라, 충분한 데이터와 컴퓨팅 자원이 있을 때 자신만의 모델을 밑바닥부터 사전 훈련하는 방법도 배웠다.

-> 트랜스포머 모델로 거의 모든 NLP 문제를 다룰 준비를 마친 셈이다.